In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

#Environnement d'exécution > Modifier le type d'exécution

Sun Sep  8 10:33:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

https://pub.towardsai.net/running-ollama-on-google-colab-free-tier-a-step-by-step-guide-9ef74b1f8f7a

In [ ]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

In [ ]:
!pip install langchain
!pip install langchain_community

In [ ]:
%xterm
#curl -fsSL https://ollama.com/install.sh | sh       #Permet de download Ollama qui nous permettra de pull nos modeles
#ollama serve                                          #Pour démarrer ollama
#ollama serve & ollama pull mixtral:8x7b      ollama serve & ollama pull llama3.1:8b
#ollama serve & ollama pull llama3.1:70b
#ollama serve & ollama pull qwen2
       #Parfois il est nécessaire de relancer la commande pour qu'il pull le modele
#Choisir son modele en fonction de ceux disponible sur https://ollama.com/library

Launching Xterm...

<IPython.core.display.Javascript object>

In [ ]:
# Import Ollama module from Langchain
from langchain_community.llms import Ollama

# Initialize an instance of the Ollama model
llm = Ollama(model="llama3.1:70b")

# Invoke the model to generate responses


In [ ]:
#response = llm.invoke("Tell me a joke")
#print(response)

KeyboardInterrupt: 

In [ ]:
%pip install pandas
%pip install llama-index-llms-ollama

In [ ]:
!pip install llama-index
%pip install ollama
import pandas as pd
import numpy as np
import os
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
import ollama


In [ ]:
#on va  juste tester le modele pour voir si il est bien fonctionnel

#resp = llm.invoke("Donne moi une recette d'omelette")
#print(resp)

In [ ]:
import pandas as pd

# Spécifiez le chemin vers votre fichier CSV
file_path = '/content/drive/MyDrive/projet de recherche/2023 fichier de base.csv'

# Lire le fichier en spécifiant le séparateur ; et un encodage approprié
try:
    df = pd.read_csv(file_path, sep=';', encoding='ISO-8859-1')  # Remplacez l'encodage si nécessaire
    print(df.head())  # Affichez les premières lignes du DataFrame
except UnicodeDecodeError as e:
    print(f"Erreur lors de la lecture du fichier : {e}")
except Exception as e:
    print(f"Une erreur s'est produite : {e}")


     Use_Case_ID Department_Code                         Agency Office  \
0  DHS-0000-2023             DHS  Customs and Border Protection    NaN   
1  DHS-0001-2023             DHS  Customs and Border Protection    NaN   
2  DHS-0002-2023             DHS  Customs and Border Protection    NaN   
3  DHS-0003-2023             DHS  Customs and Border Protection    NaN   
4  DHS-0004-2023             DHS  Customs and Border Protection    NaN   

                                         Title  \
0                    AI Curated Synthetic Data   
1      AI for Autonomous Situational Awareness   
2  Automated Item of Interest Detection - ICAD   
3                          Autonomous Aerostat   
4                Autonomous Maritime Awareness   

                                             Summary  \
0  AI Curated Synthetic Data creates synthetic da...   
1  The AI for autonomous situational awareness sy...   
2  The software analyzes photographs that are tak...   
3  Aerostat capability that us

In [ ]:
# Configuration du modèle
llm = Ollama(base_url="http://localhost:11434",
             model="llama3.1:70b",
             temperature=0.2, request_timeout=8000)

# Lecture du fichier CSV contenant les prompts
prompts_path = '/content/drive/MyDrive/projet de recherche/prompts_text1.csv'
prompts = pd.read_csv(prompts_path)

# Assurez-vous que le fichier est bien structuré
if prompts.shape[1] < 1:
    raise ValueError("Le DataFrame prompts doit avoir au moins 1 colonne.")

# Récupérer le modèle de prompt à partir de la première colonne de prompts
prompt_0 = prompts.iloc[1, 0]  # Utilisation du deuxième prompt (index 2)

# Initialisation des listes pour stocker les résultats
id_mistral = []
sent_mistral = []
text = []

# Assurez-vous que df est chargé et contient les colonnes nécessaires
# Exemple : df = pd.read_csv('your_file.csv') # À remplacer par le vrai chemin de votre fichier

# Boucle sur chaque ligne du DataFrame df
for i in range(len(df)):
    tweet_text = df.iloc[i, 5]  # Récupérer le texte de la colonne 6

    # Vérifier si tweet_text est NaN ou vide
    if pd.isna(tweet_text) or tweet_text == "":
        print(f"Ligne #{i} ignorée car le texte est manquant.")
        continue  # Sauter cette itération si le texte est manquant

    # Assurez-vous que tweet_text est une chaîne de caractères
    tweet_text = str(tweet_text)

    # Remplacer le placeholder dans le prompt
    prompt_i = prompt_0.replace('TWEET_TEXT_HERE', tweet_text)

    # Afficher le prompt_i pour chaque itération
    print(f"Prompt pour le tweet #{i} : {prompt_i}")

    try:
        response = llm.complete(prompt_i)  # Appel à l'API pour obtenir la réponse
        text.append(tweet_text)
        sent_mistral.append(response.text)
        id_mistral.append(df.iloc[i, 1])  # Récupérer l'ID ou autre identifiant
        print(f'Tweet #{i}: Traitement terminé.')
    except Exception as e:
        print(f"Erreur lors du traitement du tweet #{i}: {e}")
        sent_mistral.append("Erreur")  # Gérer les erreurs
        text.append(tweet_text)
        id_mistral.append(df.iloc[i, 1])

# Création du DataFrame final avec les résultats
df_mistral0 = pd.DataFrame(list(zip(id_mistral, text, sent_mistral)),
                           columns=['id', 'text', 'sentiment_mistral'])

df_mistral0['prompt'] = 0

# Sauvegarder le DataFrame final dans un fichier CSV
df_mistral0.to_csv('Trein_prompt.csv', index=False)

# Afficher les premières lignes du DataFrame final
print(df_mistral0.info())
print(df_mistral0.head())
print('Length mistral0:', len(df_mistral0))


Prompt pour le tweet #0 : """Please provide the target group (persons or groups whose behavior and well-being are affected by the algorithm) to whom these algorithms are addressed, as expressed by the user in the text enclosed by triple backticks. \
Give only the target group without explanation in your answer. \ Generally ambiguous texts could be classified as neutral. \ ```{AI Curated Synthetic Data creates synthetic data for computer vision to enable more capable and ethical AI when detecting anomalies in complex environments.

Specifically, it creates an emulated X-ray sensor that can produce visually realistic synthetic X-ray scan images similar to real X-ray scan images, and virtual 3D Assets of vehicles and narcotics containers. These images will be used to enhance the development of Anomaly Detection Algorithms for Non-Intrusive Inspection, incorporating AI/ML for the detection of narcotics and other contraband in conveyances and cargo.}```"""
Tweet #0: Traitement terminé.
Prom

In [ ]:
from google.colab import files

# Télécharger le fichier sur votre ordinateur local
files.download('/content/Trein_prompt.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>